In [38]:
import requests 
import pandas as pd 
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from Levenshtein import distance
from datetime import datetime

 RECALLS WEB SCRAPING SCRIPT
get_recalls_in_date_range is a function that uses the government API to scrape for recalls in the given date range. This function is used for 3 purposes
1) initially populating the database with recalls (if database was ever cleared)
2) adding new recalls to the database periodically (based on a measure put into the "num_months" variable)
delete_recalls deletes any recalls before a given data input

In [39]:
# get recalls in certain date range
# param format: yyyy-mm-dd
def get_recalls_in_date_range(start_date: str, end_date: str):

    # call API and store in pandas df
    headers = {
        'authority': 'www.etoro.com',
        'pragma': 'no-cache',
        'cache-control': 'no-cache',
        'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
        'accept': 'application/json, text/plain, */*',
        'accounttype': 'Real',
        'applicationidentifier': 'ReToro',
        'sec-ch-ua-mobile': '?0',
        'applicationversion': '331.0.2',
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'cors',
        'sec-fetch-dest': 'empty',
        'referer': 'https://www.etoro.com/discover/markets/cryptocurrencies',
        'accept-language': 'en-US,en;q=0.9',
        'cookie': '__cfruid=e7f40231e2946a1a645f6fa0eb19af969527087e-1624781498; _gcl_au=1.1.279416294.1624782732; _gid=GA1.2.518227313.    1624782732; _scid=64860a19-28e4-4e83-9f65-252b26c70796; _fbp=fb.1.1624782732733.795190273;  __adal_ca=so%3Ddirect%26me%3Dnone%26ca%3Ddirect%26co%3D%28not%2520set%29%26ke%3D%28not%2520set%29; __adal_cw=1624782733150; _sctr=1| 1624732200000; _gaexp=GAX1.2.eSuc0QBTRhKbpaD4vT_-oA.18880.x331; _hjTLDTest=1; _hjid=bb69919f-e61b-4a94-a03b-db7b1f4ec4e4;    hp_preferences=%7B%22locale%22%3A%22en-gb%22%7D; funnelFromId=38; eToroLocale=en-gb; G_ENABLED_IDPS=google;    marketing_visitor_regulation_id=10; marketing_visitor_country=96;  __cflb=0KaS4BfEHptJdJv5nwPFxhdSsqV6GxaSK8BuVNBmVkuj6hYxsLDisSwNTSmCwpbFxkL3LDuPyToV1fUsaeNLoSNtWLVGmBErMgEeYAyzW4uVUEoJHMzTirQMGVAqNKRnL ; __cf_bm=6ef9d6f250ee71d99f439672839b52ac168f7c89-1624785170-1800-ASu4E7yXfb+ci0NsW8VuCgeJiCE72Jm9uD7KkGJdy1XyNwmPvvg388mcSP    +hTCYUJvtdLyY2Vl/ekoQMAkXDATn0gyFR0LbMLl0b7sCd1Fz/Uwb3TlvfpswY1pv2NvCdqJBy5sYzSznxEsZkLznM+IGjMbvSzQffBIg6k3LDbNGPjWwv7jWq/EbDd ++xriLziA==; _uetsid=2ba841e0d72211eb9b5cc3bdcf56041f; _uetvid=2babee20d72211eb97efddb582c3c625; _ga=GA1.2.1277719802.1624782732;    _gat_UA-2056847-65=1; __adal_ses=*; __adal_id=47f4f887-c22b-4ce0-8298-37d6a0630bdd.1624782733.2.1624785174.1624782818. 770dd6b7-1517-45c9-9554-fc8d210f1d7a; _gat=1;    TS01047baf=01d53e5818a8d6dc983e2c3d0e6ada224b4742910600ba921ea33920c60ab80b88c8c57ec50101b4aeeb020479ccfac6c3c567431f;     outbrain_cid_fetch=true; _ga_B0NS054E7V=GS1.1.1624785164.2.1.1624785189.35;     TMIS2=9a74f8b353780f2fbe59d8dc1d9cd901437be0b823f8ee60d0ab36264e2503993c5e999eaf455068baf761d067e3a4cf92d9327aaa1db627113c6c3ae3b39cd5e8    ea5ce755fb8858d673749c5c919fe250d6297ac50c5b7f738927b62732627c5171a8d3a86cdc883c43ce0e24df35f8fe9b6f60a5c9148f0a762e765c11d99d;     mp_dbbd7bd9566da85f012f7ca5d8c6c944_mixpanel=%7B%22distinct_id%22%3A%20%2217a4c99388faa1-0317c936b045a4-34647600-13c680-17a4c993890d70%2    2%2C%22%24device_id%22%3A%20%2217a4c99388faa1-0317c936b045a4-34647600-13c680-17a4c993890d70%22%2C%22%24initial_referrer%22%3A%20%22%24di    rect%22%2C%22%24initial_referring_domain%22%3A%20%22%24direct%22%7D',
    }   


    url = "https://www.saferproducts.gov/RestWebServices/Recall?format=json&RecallDateStart={}&RecallDateEnd={}".format(start_date, end_date)
    print (url)
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Error: {}".format(response.status_code))
        return

    initial_df = pd.DataFrame(response.json())
    selected_fields = ['Products', 'Description',
                       'RecallDate', 'URL', 'Manufacturers', 'RemedyOptions']
    df = initial_df[selected_fields]
    return df
   
end_date = datetime.today().strftime("%Y-%m-%d")

# FORMAT: YEAR-MONTH-DAY
df = get_recalls_in_date_range("2000-01-01", end_date)

df.head()

https://www.saferproducts.gov/RestWebServices/Recall?format=json&RecallDateStart=2000-01-01&RecallDateEnd=2024-11-04


,Products,Description,RecallDate,URL,Manufacturers,RemedyOptions
0,"[{'Name': 'Tumblers with a Metal Straw', 'Desc...",This recall involves a reusable tumbler with a...,2024-10-31T00:00:00,https://www.cpsc.gov/Recalls/2025/HALO-Recalls...,[],"[{'Option': 'Refund'}, {'Option': 'Replace'}]"
1,[{'Name': 'Bauer 20V Battery-Powered 4-Gallon ...,This recall involves the Bauer 20V Cordless 4-...,2024-10-31T00:00:00,https://www.cpsc.gov/Recalls/2025/Harbor-Freig...,[],[{'Option': 'Refund'}]
2,[{'Name': 'Humane Charge Case Accessory for Ai...,This recall involves the Humane Charge Case Ac...,2024-10-31T00:00:00,https://www.cpsc.gov/Recalls/2025/Humane-Recal...,[],"[{'Option': 'Refund'}, {'Option': 'Replace'}]"
3,[{'Name': 'Manhattan Comfort Floating Wall The...,This recall involves Manhattan Comfort Cabrini...,2024-10-31T00:00:00,https://www.cpsc.gov/Recalls/2025/Manhattan-Co...,[],[{'Option': 'Repair'}]
4,[{'Name': 'Model Year 2023-2024 Ranger XP Kine...,This recall involves Model Year 2023-2024 RANG...,2024-10-31T00:00:00,https://www.cpsc.gov/Recalls/2025/Polaris-Indu...,"[{'Name': 'Polaris Industries Inc., of Medina,...",[{'Option': 'Repair'}]


Data Cleaning - Product Name / Model

In [40]:
# Extract ProductName and Model from the 'Products' dictionary
# Handle cases where 'Products' might be missing or empty
df['ProductName'] = df['Products'].apply(lambda x: x[0]['Name'] if isinstance(x, list) and len(x) > 0 else '')
df['Model'] = df['Products'].apply(lambda x: x[0].get('Model', '') if isinstance(x, list) and len(x) > 0 else '')

# Drop the original 'Products' column to clean up the DataFrame
df = df.drop('Products', axis=1)

# Display the first few rows of the DataFrame to verify
print(df[['ProductName', 'Model']].head())
# Convert the RemedyOption column to a string
# Safely handle missing or empty values
df['RemedyOptions'] = df['RemedyOptions'].apply(lambda x: ', '.join(list(x[0].values())) if isinstance(x, list) and len(x) > 0 else '')

# Display the first few rows of the DataFrame
len(df)


                                         ProductName Model
0                        Tumblers with a Metal Straw      
1  Bauer 20V Battery-Powered 4-Gallon Backpack Sp...      
2            Humane Charge Case Accessory for Ai Pin      
3  Manhattan Comfort Floating Wall Theater Entert...      
4  Model Year 2023-2024 Ranger XP Kinetic Recreat...      


7399

In [41]:

# Load the CSV file
df2 = pd.read_csv('form.csv')

# Load English stop words
stop_words = set(stopwords.words('english'))

# Ensure both Make and Model columns are lowercase
df2['Make'] = df2['Make'].str.lower()
df2['Model'] = df2['Model'].str.lower()

# Create a combined field for matching
df2['FullProduct'] = df2['Make'] + ' ' + df2['Model']

# Set to track already added rows
added_rows = set()
matches = []

# Increased threshold for stricter matching
FUZZY_THRESHOLD = 90
MAKE_THRESHOLD = 85

# Dictionary of abbreviations
abbreviation_dict = {
    'ss': 'steam shot',
    # Add more abbreviations as needed
}

def replace_abbreviations(text, abbrev_dict):
    if pd.isna(text):
        return ''
    words = text.lower().split()
    return ' '.join([abbrev_dict.get(word, word) for word in words])

def extract_make_model(text):
    """Extract potential make and model from product name"""
    if pd.isna(text):
        return '', ''
    words = text.lower().split()
    if len(words) >= 2:
        return words[0], ' '.join(words[1:])
    return text, ''

# Iterate through the rows of the main dataframe df
for index, row in df.iterrows():
    if index in added_rows:
        continue
        
    product_name = row['ProductName']
    
    # Skip if product name is NaN
    if pd.isna(product_name):
        continue
        
    # Clean and normalize the product name
    product_name = replace_abbreviations(product_name.lower(), abbreviation_dict)
    potential_make, potential_model = extract_make_model(product_name)
    
    found_match = False
    
    # Iterate through df2 rows
    for idx2, row2 in df2.iterrows():
        make = row2['Make']
        model = row2['Model']
        full_product = row2['FullProduct']
        
        # Check make match first
        make_ratio = fuzz.ratio(potential_make, make)
        
        if make_ratio >= MAKE_THRESHOLD:
            # Only check model if make matches
            model_ratio = fuzz.partial_ratio(potential_model, model)
            
            # Both make and model need to match with high confidence
            if make_ratio >= MAKE_THRESHOLD and model_ratio >= FUZZY_THRESHOLD:
                match_data = row.to_dict()
                match_data.update({
                    'Matched_Make': row2['Make'],
                    'Matched_Model': row2['Model'],
                    'Matched_Year': row2['PurchaseYear'],
                    'Make_Confidence': make_ratio,
                    'Model_Confidence': model_ratio
                })
                matches.append(match_data)
                added_rows.add(index)
                found_match = True
                break
    

# Convert matches to DataFrame
matches_df = pd.DataFrame(matches)

Data Cleaning - RemedyOptions

In [42]:
#match CSV to .gov
# hammign distance recognizing different spellings
# Incorporate month year from retailers to find in database from input
# RecallID, Date of Recall, Product Name, Remedy Options
# Check the text distance of manufacturer in the title
# If algorithm is very confident, then return only one option
# Create confidence threshold constant
# Sort based on date ranges on higher priority then iterate for name and make
# Sort all the date stuff before hand and store in a simpler file for accessing claims
#XML Json Gson
# Save matches to JSON file
json_filename = 'product_matches.json'

# Convert DataFrame to JSON format
# orient='records' will create a list of dictionaries
matches_df.to_json(json_filename, orient='records', indent=4)

print(f"Matches have been saved to {json_filename}")

Matches have been saved to product_matches.json
